In [1]:
room_config = {
    "officeroom": {
        "north": "wall",
        "east": "livingroom",
        "south": "wall",
        "west": "wall",
    },
    "livingroom": {
        "north": "wall",
        "east": "wall",
        "south": "bedroom",
        "west": "officeroom",
    },
    "bedroom": {
        "north": "livingroom",
        "east": "wall",
        "south": "wall",
        "west": "wall",
    },
}


object_transition_config = {
    "static": {"bed": None, "desk": None, "table": None},
    "independent": {
        "tae": {
            "officeroom": {"north": 0, "east": 0.1, "south": 0, "west": 0, "stay": 0.9},
            "livingroom": {
                "north": 0,
                "east": 0,
                "south": 0,
                "west": 0.1,
                "stay": 0.9,
            },
            "bedroom": {"north": 0, "east": 0, "south": 0, "west": 0, "stay": 0},
        },
        "michael": {
            "officeroom": {
                "north": 0,
                "east": 0,
                "south": 0,
                "west": 0,
                "stay": 0,
            },
            "livingroom": {
                "north": 0,
                "east": 0,
                "south": 0.9,
                "west": 0,
                "stay": 0.1,
            },
            "bedroom": {"north": 0.1, "east": 0, "south": 0, "west": 0, "stay": 0.9},
        },
        "vincent": {
            "officeroom": {
                "north": 0,
                "east": 0.5,
                "south": 0,
                "west": 0,
                "stay": 0.5,
            },
            "livingroom": {
                "north": 0,
                "east": 0,
                "south": 0.333,
                "west": 0.333,
                "stay": 0.333,
            },
            "bedroom": {
                "north": 0.5,
                "east": 0,
                "south": 0,
                "west": 0,
                "stay": 0.5,
            },
        },
    },
    "dependent": {
        "laptop": {"tae": 0.7, "michael": 0.4, "vincent": 0.1},
        "phone": {"tae": 0.1, "michael": 0.7, "vincent": 0.4},
        "headset": {"tae": 0.4, "michael": 0.1, "vincent": 0.9},
    },
    "agent": {
        "agent": {"officeroom": None, "livingroom": None, "bedroom": None},
    },
}

object_init_config = {
    "static": {
        "bed": {"officeroom": 0, "livingroom": 0, "bedroom": 1},
        "desk": {"officeroom": 1, "livingroom": 0, "bedroom": 0},
        "table": {"officeroom": 0, "livingroom": 1, "bedroom": 0},
    },
    "independent": {
        "tae": {"officeroom": 0.5, "livingroom": 0.5, "bedroom": 0},
        "michael": {"officeroom": 0, "livingroom": 0.5, "bedroom": 0.5},
        "vincent": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
    },
    "dependent": {
        "laptop": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
        "phone": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
        "headset": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
    },
    "agent": {
        "agent": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
    },
}

config = {
    "room_config": room_config,
    "object_transition_config": object_transition_config,
    "object_init_config": object_init_config,
    "question_prob": 1.0,
    "seed": 42,
    "terminates_at": 99,
}

In [2]:
import gymnasium as gym
import random

env = gym.make("room_env:RoomEnv-v2", **config)
(obs, question), info = env.reset()
while True:
    action_qa = question[0]
    action_explore = random.choice(["north", "east", "south", "west", "stay"])
    (obs, question), reward, done, truncated, info = env.step(("wall", action_explore))
    if done:
        break

/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/envs/registration.py:491: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes']
  logger.warn(
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:197: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:210: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:156: UserWarning: WARN: The obs returned by the `res

In [3]:
env.step()

AttributeError: 'RoomEnv2' object has no attribute 'stepp'

In [ ]:
env.answers

In [ ]:
from room_env.envs.room2 import (
    Object,
    StaticObject,
    IndepdentObject,
    DependentObject,
    Room,
    RoomEnv2,
)

env = RoomEnv2(**config)
(obs, question), info = env.reset()

rewards = []
while True:
    (obs, question), reward, done, info = env.step("wall", "north")
    print(obs, question, reward, done, info)
    rewards.append(reward)
    if done:
        break

In [ ]:
from room_env.envs.room2 import (
    Object,
    StaticObject,
    IndepdentObject,
    DependentObject,
    Room,
    RoomEnv2,
)

env = RoomEnv2(**config)
(obs, question), info = env.reset()

obs, question

In [ ]:
rewards = []
while True:
    (obs, question), reward, done, info = env.step("wall", "north")
    print(obs, question, reward, done, info)
    rewards.append(reward)
    if done:
        break

In [ ]:
len(rewards)

In [ ]:
env.question

In [ ]:
env._get_hidden_global_state()

In [ ]:
(obs, question), reward, done, info = env.step("wall", "north")
(obs, question), reward, done, info